In [ ]:
%reload_kedro

In [ ]:
import numpy as np
from sklearn import linear_model, decomposition
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler

In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
X_train = catalog.load("master_table_treino_ftes")
y_train = catalog.load("master_table_treino_tgt")

In [ ]:
std_slc = StandardScaler()
pca = decomposition.PCA()
logistic_Reg = linear_model.LogisticRegression()

In [ ]:
pipe = Pipeline(steps=[('std_slc', std_slc),
                        ('pca', pca),
                        ('logistic_Reg', logistic_Reg)])

In [ ]:
n_components = list(range(1,X_train.shape[1]+1,1))

In [ ]:
C = np.logspace(-4, 4, 50)
penalty = ['l1', 'l2', "elasticnet"]

In [ ]:
parameters = dict(pca__n_components=n_components,
                logistic_Reg__C=C,
                logistic_Reg__penalty=penalty)

In [ ]:
clf_GS = GridSearchCV(pipe, parameters)
clf_GS.fit(X_train, y_train)

In [ ]:
print('Best Penalty:', clf_GS.best_estimator_.get_params()['logistic_Reg__penalty'])
print('Best C:', clf_GS.best_estimator_.get_params()['logistic_Reg__C'])
print('Best Number Of Components:', clf_GS.best_estimator_.get_params()['pca__n_components'])
print(); print(clf_GS.best_estimator_.get_params()['logistic_Reg'])

In [ ]:
cv_res = cross_val_score(clf_GS, X_train, y_train, cv=4, n_jobs=-1)
print(cv_res)
print(cv_res.mean())
print(cv_res.std())